In [2]:
!pip install bert-score comet-ml

In [3]:
!pip install ace_tools

In [4]:
import pandas as pd
import json
from bert_score import score as bert_score
from comet import download_model, load_from_checkpoint

# ✅ 파일 경로 설정
TEST_RESULTS_PATH = "test_results.csv"
EVAL_RESULTS_PATH = "evaluation_results.csv"

# ✅ 저장된 번역 결과 로드
df_results = pd.read_csv(TEST_RESULTS_PATH)

# ✅ NaN 값 처리 (빈 값을 문자열로 변환)
df_results["generated_output"] = df_results["generated_output"].fillna("").astype(str)
df_results["expected_output"] = df_results["expected_output"].fillna("").astype(str)


In [5]:
# ✅ 언어 코드 매핑 (자동 감지)
LANG_CODE_MAP = {
    "KO": "ko",
    "EN": "en",
    "JPN": "ja"
}

In [6]:
# ✅ BERTScore 평가 함수
def compute_bertscore(df, lang_code):
    if df.empty:
        return None, None, None

    P, R, F1 = bert_score(
        df["generated_output"].tolist(),
        df["expected_output"].tolist(),
        lang=lang_code,
        verbose=True
    )
    return P, R, F1

# ✅ BERTScore 평가 수행
print("\n🔍 BERTScore 평가 진행 중...")
target_lang = df_results["target_language"].iloc[0]  # 첫 번째 샘플 기준으로 언어 감지
lang_code = LANG_CODE_MAP.get(target_lang, "en")  # 기본값 EN
P, R, F1 = compute_bertscore(df_results, lang_code=lang_code)

# ✅ 평가 점수 DataFrame에 추가
df_results["BERTScore_P"] = P
df_results["BERTScore_R"] = R
df_results["BERTScore_F1"] = F1

print(f"✅ BERTScore 평가 완료! (사용된 언어 코드: {lang_code})")


🔍 BERTScore 평가 진행 중...
calculating scores...
computing bert embedding.


  0%|          | 0/152 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/85 [00:00<?, ?it/s]

done in 24.41 seconds, 221.89 sentences/sec
✅ BERTScore 평가 완료! (사용된 언어 코드: ko)


In [7]:
# ✅ COMET 모델 로드
print("\n🔍 COMET 평가 모델 로드 중...")
comet_model_path = download_model("Unbabel/wmt22-cometkiwi-da")
comet_model = load_from_checkpoint(comet_model_path)

# ✅ COMET 평가 함수
def compute_comet(df):
    if df.empty:
        return None

    data = [
        {"src": inp, "mt": gen, "ref": ref}
        for inp, gen, ref in zip(df["input"], df["generated_output"], df["expected_output"])
    ]
    scores = comet_model.predict(data, batch_size=8, gpus=1)["scores"]
    return scores

# ✅ COMET 평가 수행
print("\n🔍 COMET 평가 진행 중...")
df_results["COMET"] = compute_comet(df_results)
print("✅ COMET 평가 완료!")

# ✅ 기존 형식 유지하면서 평가 점수 추가하여 저장
df_results.to_csv(EVAL_RESULTS_PATH, index=False, encoding="utf-8-sig")
print(f"\n✅ 평가 결과가 저장되었습니다: {EVAL_RESULTS_PATH}")


🔍 COMET 평가 모델 로드 중...


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\201\.cache\huggingface\hub\models--Unbabel--wmt22-cometkiwi-da\snapshots\a79bfca292fc1689cbab6ef941ca6fc439f7195f\checkpoints\model.ckpt`
Encoder model frozen.
c:\Users\201\miniforge3\envs\pystudy\Lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_


🔍 COMET 평가 진행 중...


Predicting DataLoader 0: 100%|██████████| 677/677 [00:57<00:00, 11.82it/s]


✅ COMET 평가 완료!

✅ 평가 결과가 저장되었습니다: evaluation_results.csv


In [9]:
import pandas as pd
from IPython.display import display

# ✅ 평균 평가 점수 계산
mean_bertscore_p = df_results["BERTScore_P"].mean()
mean_bertscore_r = df_results["BERTScore_R"].mean()
mean_bertscore_f1 = df_results["BERTScore_F1"].mean()
mean_comet = df_results["COMET"].mean()

# ✅ 평가 결과 출력
print("\n📊 번역 평가 결과 요약 📊")
print(f"🔹 평균 BERTScore (Precision): {mean_bertscore_p:.4f}")
print(f"🔹 평균 BERTScore (Recall): {mean_bertscore_r:.4f}")
print(f"🔹 평균 BERTScore (F1 Score): {mean_bertscore_f1:.4f}")
print(f"🔹 평균 COMET 점수: {mean_comet:.4f}")

# ✅ Jupyter Notebook에서 결과 표시
display(df_results)


📊 번역 평가 결과 요약 📊
🔹 평균 BERTScore (Precision): 0.8977
🔹 평균 BERTScore (Recall): 0.8955
🔹 평균 BERTScore (F1 Score): 0.8965
🔹 평균 COMET 점수: 0.8275


,input,expected_output,generated_output,target_language,glossary,BERTScore_P,BERTScore_R,BERTScore_F1,COMET
0,I decided to take a walk with my friends at Gu...,구봉산 근린공원에서 친구들과 산책을 하기로 했어요.,구봉산 근린공원에서 친구들과 함께 산책을 하기로 했어요.,KO,"{'ENG': 'Gubongsan Neighborhood Park', 'JPN': ...",0.981155,0.988651,0.984889,0.895836
1,원적산의 경치는 정말 아름다워요.,The scenery of Wonjeok Mountain is truly beaut...,The scenery of Wonjeok Mountain is truly beaut...,ENG,"{'ENG': 'Wonjeok Mountain', 'JPN': '原積算', 'KO'...",1.000000,1.000000,1.000000,0.832988
2,The movie filmed at the Iksan Prison Film Set ...,익산 교도소 세트장에서 촬영한 영화가 큰 인기를 끌고 있다.,익산 교도소 세트장을 사용한 영화는 많은 인기를 끌고 있다.,KO,"{'ENG': 'Iksan Prison Film Set', 'JPN': '益山刑務所...",0.930290,0.920392,0.925315,0.886820
3,"여러분, 한국의 매력적인 음식 문화 속에서 '오징어두루치기'를 통해 신선한 해산물과...",皆さん、韓国の魅力的な食文化の中で「イカの野菜炒め」を通じて新鮮な海鮮と辛い調味料が調和した...,皆さん、韓国の魅力的な食文化の中で「イカの野菜炒め」を通じて、新鮮な海産物と辛いコチュジャン...,JPN,"{'ENG': 'Stir-fried Squid', 'JPN': 'イカの野菜炒め', ...",0.950356,0.972483,0.961292,0.747779
4,고석정 꽃밭은 아름다운 자연경관과 다채로운 꽃들이 어우러져 여행자 여러분을 환상적인...,Goseokjeong Flower Garden is a place that invi...,Goseokjeong Flower Garden is a destination tha...,ENG,"{'ENG': 'Goseokjeong Flower Garden', 'JPN': '孤...",0.986065,0.980333,0.983191,0.841979
...,...,...,...,...,...,...,...,...,...
5411,サリゴル公園は自然の美しさと調和した休息所であり、訪問者に平和な散歩道とさまざまな野生花を鑑...,"싸리골공원은 자연의 아름다움과 조화롭게 어우러진 휴식처로, 방문객들에게 평화로운 산...","sassi gol park은 자연의 아름다움과 조화롭게 어우러진 휴식처로, 방문객들...",KO,"{'ENG': 'Sarigol Park', 'JPN': 'サリゴル公園', 'KO':...",0.949561,0.953333,0.951443,0.880476
5412,봉원사에 가면 아름다운 풍경을 감상할 수 있습니다.,奉元寺に行くと、美しい景色を楽しむことができます。,"If you go to Bongwonsa Temple, you can enjoy b...",JPN,"{'ENG': 'Bongwonsa Temple', 'JPN': '奉元寺', 'KO'...",0.700832,0.689508,0.695124,0.838903
5413,오늘은 산들공원에서 산책할 거예요.,"Today, I will take a walk in Sandeul Park.","Today, I will take a walk in Sandeul Park.",ENG,"{'ENG': 'Sandeul Park', 'JPN': 'サンドゥル公園', 'KO'...",1.000000,1.000000,1.000000,0.839124
5414,I took a walk with a friend at Dogok Neighborh...,도곡근린공원에서 친구와 함께 산책을 했어요.,도곡근린공원에서 친구와 함께 산책을 했어요.,KO,"{'ENG': 'Dogok Neighborhood Park', 'JPN': '道谷近...",1.000000,1.000000,1.000000,0.892391
